# New Section

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd

df_products = pd.read_csv("product_info.csv",
                 dtype=str, low_memory=False)

df_products.head()
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8494 entries, 0 to 8493
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   product_id          8494 non-null   object
 1   product_name        8494 non-null   object
 2   brand_id            8494 non-null   object
 3   brand_name          8494 non-null   object
 4   loves_count         8494 non-null   object
 5   rating              8216 non-null   object
 6   reviews             8216 non-null   object
 7   size                6863 non-null   object
 8   variation_type      7050 non-null   object
 9   variation_value     6896 non-null   object
 10  variation_desc      1250 non-null   object
 11  ingredients         7549 non-null   object
 12  price_usd           8494 non-null   object
 13  value_price_usd     451 non-null    object
 14  sale_price_usd      270 non-null    object
 15  limited_edition     8494 non-null   object
 16  new                 8494

In [3]:
# Read reviews
df_reviews_1 = pd.read_csv("reviews_0-250.csv",index_col = 0, dtype={'author_id':'str'})
df_reviews_5 = pd.read_csv("reviews_1250-end.csv",index_col = 0, dtype={'author_id':'str'})
df_reviews_2 = pd.read_csv("reviews_250-500.csv",index_col = 0, dtype={'author_id':'str'})
df_reviews_3 = pd.read_csv("reviews_500-750.csv",index_col = 0, dtype={'author_id':'str'})
df_reviews_4 = pd.read_csv("reviews_750-1250.csv",index_col = 0, dtype={'author_id':'str'})

In [4]:
df_reviews = pd.concat([df_reviews_1,df_reviews_2,df_reviews_3,df_reviews_4,df_reviews_5],axis=0)

In [5]:
df = pd.merge(df_reviews, df_products, how='outer', on=['product_id', 'product_id'])
df.shape

(1100554, 44)

In [6]:
## Drop columns
df = df.drop(['variation_type', 'variation_value', 'variation_desc', 'value_price_usd', 'sale_price_usd', 'sephora_exclusive', 'child_count',
              'child_max_price', 'child_min_price', 'submission_time', 'review_title', 'online_only'], axis=1)
df.shape

(1100554, 32)

In [7]:
## Find missing values
## Asta e luat de undeva
missing = []
unique = []
types = []
variables = []
count = []

for item in df.columns:
    variables.append(item)
    missing.append(df[item].isnull().sum())
    unique.append(df[item].nunique())
    types.append(df[item].dtypes)
    count.append(len(df[item]))

output = pd.DataFrame({
    'variable': variables,
    'dtype': types,
    'count': count,
    'unique': unique,
    'missing': missing,

})
print("Information products : ")
output.sort_values("missing",ascending=False).reset_index(drop=True)


Information products : 


,variable,dtype,count,unique,missing
0,helpfulness,float64,1100554,3767,567735
1,hair_color,object,1100554,7,232911
2,eye_color,object,1100554,6,215771
3,skin_tone,object,1100554,14,176682
4,is_recommended,float64,1100554,2,174131
5,tertiary_category,object,1100554,118,161894
6,skin_type,object,1100554,4,117700
7,highlights,object,1100554,4417,115729
8,size,object,1100554,2055,44661
9,ingredients,object,1100554,6538,22843


In [8]:
df.dropna(axis=0,inplace=True)
df.shape

(312075, 32)

In [12]:

## Aici doar m-am uitat sa vad la grupul pt 1 produs
grouped = df.groupby("product_id")
pid = df["product_id"].iloc[0]
one_group = grouped.get_group(pid)
print(one_group)

       author_id  rating_x  is_recommended  helpfulness  total_feedback_count  \
277   2206599853       5.0             1.0     1.000000                   2.0   
278   1755055004       5.0             1.0     0.666667                   3.0   
279  21407018770       5.0             1.0     1.000000                   1.0   
280   6645877566       5.0             1.0     1.000000                   3.0   
282    993395592       5.0             1.0     1.000000                   6.0   
..           ...       ...             ...          ...                   ...   
623   5690932785       5.0             1.0     1.000000                   1.0   
639   1306081161       5.0             1.0     1.000000                   7.0   
641   5375313577       4.0             1.0     1.000000                   1.0   
646   5856779272       1.0             0.0     1.000000                   2.0   
649   2844485529       5.0             1.0     1.000000                   1.0   

     total_neg_feedback_cou

# **MAKE SKIN TYPE COLL**

In [9]:
from collections import defaultdict, Counter

## Nu le combin ci adaug camp cu skin in produse conform mediei utilizatorilor pozitivi
df_ponderat = df
df_ponderat["weight"] = df["is_recommended"] * df["total_pos_feedback_count"]

## face dictionar cu d[id][skin] ca sa calc cel mai mare weight
product_skin_scores = defaultdict(lambda: Counter())

for _, row in df_ponderat.iterrows():
    pid = row["product_id"]
    skin = row["skin_type"]
    w = row["weight"]
    if skin and w > 0:
        product_skin_scores[pid][skin] += w



In [10]:
def get_assigned_skin(product_id, product_skin_scores):
    counter = product_skin_scores.get(product_id, None)
    if counter and len(counter) > 0:
        return counter.most_common(1)[0][0]
    else:
        return None
df["assigned_skin_type"] = df["product_id"].apply(
    lambda pid: get_assigned_skin(pid, product_skin_scores)
)



In [11]:
print(df.head(3))

       author_id  rating_x  is_recommended  helpfulness  total_feedback_count  \
277   2206599853       5.0             1.0     1.000000                   2.0   
278   1755055004       5.0             1.0     0.666667                   3.0   
279  21407018770       5.0             1.0     1.000000                   1.0   

     total_neg_feedback_count  total_pos_feedback_count  \
277                       0.0                       2.0   
278                       1.0                       2.0   
279                       0.0                       1.0   

                                           review_text    skin_tone eye_color  \
277  I wanted to try these to help with the texture...  lightMedium     brown   
278  I’ve only been using this product for a few we...       medium     brown   
279  ￼ Ever since I’ve started using this, I’ve had...        light     green   

     ... price_usd_y limited_edition new out_of_stock  \
277  ...          48               0   0            0   

In [12]:
df_product = df_products[df_products["product_id"].isin(df["product_id"])].copy()

In [13]:
df_unique = df.drop_duplicates(subset=["product_id"])
df_product["assigned_skin_type"] = df_product["product_id"].map(
    df_unique.set_index("product_id")["assigned_skin_type"]
)
print(df_product.info)

<bound method DataFrame.info of      product_id                                       product_name brand_id  \
89      P439055               GENIUS Sleeping Collagen Moisturizer     6018   
90      P421277                       GENIUS Liquid Collagen Serum     6018   
93      P311143  SUBLIME DEFENSE Ultra Lightweight UV Defense F...     6018   
94      P384537                   GENIUS Ultimate Anti-Aging Cream     6018   
100     P392945        GENIUS Ultimate Anti-Aging Vitamin C+ Serum     6018   
...         ...                                                ...      ...   
8419    P416923         Superfood Hydrate + Firm Peptide Eye Cream     6169   
8420    P482329                  Retinal + Niacinamide Youth Serum     6169   
8421    P502479  Kombucha + 10% AHA Liquid Exfoliant with Lacti...     6169   
8423    P504313  Peptides + C Energy Eye Concentrate with Vitam...     6169   
8426    P503872     Mini Mandelic Acid + Superfood Unity Exfoliant     6169   

               bran

In [20]:
## Save dataset
import csv

df_product.to_csv("store_products.csv", index=False)

In [14]:
## Functie care face un alt dataset cu produse care au intr-un camp anume un anumit string
## de exemplu le iau doar pe cele care au collagen
import ast

def filter_by_highlight(df, column="highlights", keyword=""):
    def contains_keyword(x):
        try:
            if isinstance(x, str):
                lst = ast.literal_eval(x)
            elif isinstance(x, list):
                lst = x
            else:
                return False
        except:
            return False

        return any(keyword.lower() in str(i).lower() for i in lst)

    filtered_df = df[df[column].apply(contains_keyword)].copy()
    return filtered_df

In [25]:
def filter(df, column="highlights", keyword=""):
    def contains_keyword(x):
        if isinstance(x, list):
            return any(keyword.lower() in str(i).lower() for i in x)

        if isinstance(x, str):
            try:
                parsed = ast.literal_eval(x)
                if isinstance(parsed, list):
                    return any(keyword.lower() in str(i).lower() for i in parsed)
            except:
                return keyword.lower() in x.lower()

        return False
    return df[df[column].apply(contains_keyword)].copy()

In [16]:
df_filtered = filter_by_highlight(df_product, column="highlights", keyword="Hypoallergenic")

print(df_filtered)

     product_id                                       product_name brand_id  \
89      P439055               GENIUS Sleeping Collagen Moisturizer     6018   
90      P421277                       GENIUS Liquid Collagen Serum     6018   
93      P311143  SUBLIME DEFENSE Ultra Lightweight UV Defense F...     6018   
94      P384537                   GENIUS Ultimate Anti-Aging Cream     6018   
100     P392945        GENIUS Ultimate Anti-Aging Vitamin C+ Serum     6018   
106     P453818                     GENIUS Collagen Calming Relief     6018   
107     P442859                     ALIVE Prebiotic Balancing Mask     6018   
108     P388262               GENIUS Ultimate Anti-Aging Eye Cream     6018   
109     P457694     Blue Algae Vitamin C Dark Spot Correcting Peel     6018   
110     P447504                  AA (Alguronic Acid) Barrier Serum     6018   
114     P433469                     ELEVATE Advanced Retinol Serum     6018   
2102    P397310        PoreDermabrasion Pore Perfect

# **RECOMANDARE**

In [17]:
import re
## Normalizam coloanele
def normalize_text(text):
    text = re.sub(r'<.*?>', '', str(text))
    text = text.encode("ascii", "ignore").decode()
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

In [18]:
## Functie care ia un pd,id,  col1, col2 si returneaza produs similar
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_recommandation(df, col1="highlights",col2="ingredients", id = ""):
  df_exp = df
  df_exp[col1] = df_exp[col1].apply(normalize_text)
  df_exp[col2] = df_exp[col2].apply(normalize_text)

  df_exp["final_description"] = (
  df_exp[col1].fillna('') + " " + df_exp[col2].fillna('')
  ).str.strip()

  df_exp["final_description"] = df_exp["final_description"].apply(normalize_text)
  df_exp = df_exp.drop_duplicates(subset=["final_description"])


  df_exp = df_exp[df_exp["final_description"].str.strip() != ""]
  df_exp = df_exp.drop_duplicates(subset=["final_description"])


  tfidf = TfidfVectorizer(stop_words='english')
  tfidf_matrix = tfidf.fit_transform(df_exp["final_description"])

  cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

  produs_index = df_exp.index.get_loc(df_exp[df_exp["product_id"] == id].index[0])

  ## similaritate produs dorit cu celelalte
  similarities = cosine_sim[produs_index]
  similarities[produs_index] = 0

  max_score = 0
  best_index = None

  for j in range(len(similarities)):
      if similarities[j] > max_score and similarities[j] < 0.9:
          max_score = similarities[j]
          best_index = j
  return best_index

In [ ]:
df_exp = df_product

res = get_recommandation(df_exp, "ingredients", "highlights", "P311143")
produs_index = df_exp.index[df_exp["product_id"] == "P311143"].tolist()[0]

if res is not None:
    print("Produs 1:", df_exp.iloc[produs_index]["product_name"])
    print("Produs 1 description:", df_exp.iloc[produs_index]["final_description"])
    print("Produs recomandat:", df_exp.iloc[res]["product_name"])
    print("Produs 2 description:", df_exp.iloc[res]["final_description"])

Produs 1: Vinopure Pore Purifying Gel Cleanser
Produs 1 description: ['aqua/water/eau, vitis vinifera (grape) fruit water, decyl glucoside, glycerin, sodium cocoyl glutamate, sucrose cocoate, acacia senegal gum, xanthan gum, propylene glycol, citric acid, aloe barbadensis leaf powder, salicylic acid, vitis vinifera (grape) juice, sodium benzoate, pelargonium graveolens flower oil, mentha piperita (peppermint) oil, citral, cymbopogon citratus leaf oil, rosmarinus officinalis (rosemary) leaf oil, citronellol, geraniol, potassium sorbate, linalool, lavandula hybrida oil, melissa officinalis leaf oil.'] ['vegan', 'good for: pores', 'clean + planet positive', 'fragrance free', 'good for: acne/blemishes', 'best for oily, combo, normal skin']
Produs recomandat: Mini Alpha Beta AHA/BHA Daily Cleansing Gel
Produs 2 description: ['water, glycerin, propanediol, sodium lauroyl methyl isethionate, cocamidopropyl hydroxysultaine, sodium cocoamphoacetate, disteareth-75 ipdi, glycereth-7 caprylate/cap

In [ ]:
print(df_filtered.head(19))

     product_id                                       product_name brand_id  \
89      P439055               GENIUS Sleeping Collagen Moisturizer     6018   
90      P421277                       GENIUS Liquid Collagen Serum     6018   
93      P311143  SUBLIME DEFENSE Ultra Lightweight UV Defense F...     6018   
94      P384537                   GENIUS Ultimate Anti-Aging Cream     6018   
100     P392945        GENIUS Ultimate Anti-Aging Vitamin C+ Serum     6018   
106     P453818                     GENIUS Collagen Calming Relief     6018   
107     P442859                     ALIVE Prebiotic Balancing Mask     6018   
108     P388262               GENIUS Ultimate Anti-Aging Eye Cream     6018   
109     P457694     Blue Algae Vitamin C Dark Spot Correcting Peel     6018   
110     P447504                  AA (Alguronic Acid) Barrier Serum     6018   
114     P433469                     ELEVATE Advanced Retinol Serum     6018   
2102    P397310        PoreDermabrasion Pore Perfect

In [ ]:

df_filtered = filter(df_product, column="highlights", keyword="hypoallergenic")
df_filtered = filter(df_filtered, column="secondary_category", keyword="Treatments")
print(df_filtered.head(10))

     product_id                                       product_name brand_id  \
90      P421277                       GENIUS Liquid Collagen Serum     6018   
100     P392945        GENIUS Ultimate Anti-Aging Vitamin C+ Serum     6018   
109     P457694     Blue Algae Vitamin C Dark Spot Correcting Peel     6018   
110     P447504                  AA (Alguronic Acid) Barrier Serum     6018   
114     P433469                     ELEVATE Advanced Retinol Serum     6018   
2116    P453822  Dark Spots No More Triple Acid Spot Minimizing...     4372   
4440    P482692                   Water Bank Blue Hyaluronic Serum     6125   
4443    P503867  Retinol Firming Cream Treatment with Peptides ...     6125   
7710    P483697                  Firming Vegan Collagen Prep Serum     6488   
7948    P475204  Sensitive Skin Treatment Drops Calming Vitamin...     6373   

               brand_name loves_count  rating reviews                 size  \
90               Algenist       67870  4.0259    115

In [ ]:
## Incerc pt un produs sa gasesc doar cele hypoallergenice similare
df_filtered = filter(df_product, column="highlights", keyword="hypoallergenic")
df_filtered = filter(df_filtered, column="secondary_category", keyword="Treatments")

df_exp = df_filtered

res = get_recommandation(df_exp, "ingredients", "highlights", "P433469")
produs_index = df_exp.index.get_loc(df_exp[df_exp["product_id"] == "P433469"].index[0])

if res is not None:
    print("Produs 1:", df_exp.iloc[produs_index]["product_name"])
    print("Produs 1 description:", df_exp.iloc[produs_index]["final_description"])
    print("Produs recomandat:", df_exp.iloc[res]["product_name"])
    print("Produs 2 description:", df_exp.iloc[res]["final_description"])

Produs 1: ELEVATE Advanced Retinol Serum
Produs 1 description: ['water (aqua, eau), dimethicone, glycerin, butylene glycol, pentylene glycol, polymethyl methacrylate, dimethicone crosspolymer, retinol, parachlorella beijerinckii exopolysaccharides, dunaliella salina extract, ceramide np, myristoyl nonapeptide-3, palmitoyl tripeptide-1, palmitoyl tetrapeptide-7, acetyl dipeptide-1 cetyl ester, glycine soja (soybean) extract, tocopherol, squalane, acrylates/c10-30 alkyl acrylate crosspolymer, caprylic/capric triglyceride, hydroxyethyl acrylate/sodium acryloyldimethyl taurate copolymer, sodium pca, polysorbate 20, dipotassium glycyrrhizate, sorbitan laurate, dimethicone/peg-10/15 crosspolymer, sodium hydroxide, cetyl palmitate, hydrogenated polydecene, laureth-23, polysorbate 60, disodium edta, hydroxyethylcellulose, carbomer, sodium lactate, sorbitan isostearate, trideceth-6 phosphate, dipropylene glycol, sodium citrate, mica, caprylyl glycol, chlorphenesin, phenoxyethanol, fragrance (pa

# **TOP N PRODUSE**

In [19]:
## Functie care ia un pd,id,  col1, col2, n si returneaza top Nproduse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_n_recommandation(df, col1="highlights",col2="ingredients", id = "", N=1):
  df_exp = df
  df_exp[col1] = df_exp[col1].apply(normalize_text)
  df_exp[col2] = df_exp[col2].apply(normalize_text)

  df_exp["final_description"] = (
  df_exp[col1].fillna('') + " " + df_exp[col2].fillna('')
  ).str.strip()

  df_exp["final_description"] = df_exp["final_description"].apply(normalize_text)
  df_exp = df_exp.drop_duplicates(subset=["final_description"])


  df_exp = df_exp[df_exp["final_description"].str.strip() != ""]
  df_exp = df_exp.drop_duplicates(subset=["final_description"])


  tfidf = TfidfVectorizer(stop_words='english')
  tfidf_matrix = tfidf.fit_transform(df_exp["final_description"])

  cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

  produs_index = df_exp.index.get_loc(df_exp[df_exp["product_id"] == id].index[0])

  ## similaritate produs dorit cu celelalte
  similarities = cosine_sim[produs_index]
  similarities[produs_index] = 0

  max_score = 0
  similarities[produs_index] = -1
  top_indices = np.argsort(similarities)[::-1]

  return top_indices[:N]

In [20]:
## Incerc pt un produs sa gasesc doar cele hypoallergenice similare
df_filtered = filter(df_product, column="highlights", keyword="hypoallergenic")
df_filtered = filter(df_filtered, column="secondary_category", keyword="Treatments")

df_exp = df_filtered

res = get_n_recommandation(df_exp, "ingredients", "highlights", "P433469", 30)
produs_index = df_exp.index.get_loc(df_exp[df_exp["product_id"] == "P433469"].index[0])

print("Produs 1:", df_exp.iloc[produs_index]["product_name"])

for i in range(len(res)):
  print("Produs ", i+2,  df_exp.iloc[res[i]]["product_name"], "Tip piele", df_exp.iloc[res[i]]["assigned_skin_type"])


Produs 1: ELEVATE Advanced Retinol Serum
Produs  2 GENIUS Ultimate Anti-Aging Vitamin C+ Serum Tip piele combination
Produs  3 AA (Alguronic Acid) Barrier Serum Tip piele combination
Produs  4 Sensitive Skin Treatment Drops Calming Vitamin B Serum Tip piele combination
Produs  5 Retinol Firming Cream Treatment with Peptides + Hyaluronic Acid Tip piele combination
Produs  6 GENIUS Liquid Collagen Serum Tip piele combination
Produs  7 Water Bank Blue Hyaluronic Serum Tip piele combination
Produs  8 Dark Spots No More Triple Acid Spot Minimizing Concentrate Tip piele combination
Produs  9 Firming Vegan Collagen Prep Serum Tip piele combination
Produs  10 Blue Algae Vitamin C Dark Spot Correcting Peel Tip piele combination
Produs  11 ELEVATE Advanced Retinol Serum Tip piele combination


In [21]:
print(df_product["assigned_skin_type"].value_counts())
print(df_product.size)

assigned_skin_type
combination    1313
dry             112
normal           43
oily             32
Name: count, dtype: int64
42168


In [24]:
## functie care face flitru pt sa nu contina
def filter_out(df, column="highlights", keyword=""):
    def contains_keyword(x):
        if isinstance(x, list):
            return any(keyword.lower() not in str(i).lower() for i in x)

        if isinstance(x, str):
            try:
                parsed = ast.literal_eval(x)
                if isinstance(parsed, list):
                    return any(keyword.lower() not in str(i).lower() for i in parsed)
            except:
                return keyword.lower() not in x.lower()

        return True
    return df[df[column].apply(contains_keyword)].copy()

In [27]:
df_filtered = filter_out(df_product, column="highlights", keyword="hypoallergenic")

df_exp = df_filtered

res = get_n_recommandation(df_exp, "ingredients", "highlights", "P433469", 30)
produs_index = df_exp.index.get_loc(df_exp[df_exp["product_id"] == "P433469"].index[0])

print("Produs 1:", df_exp.iloc[produs_index]["product_name"])

for i in range(len(res)):
  print("Produs ", i+2,  df_exp.iloc[res[i]]["product_name"], "Tip piele", df_exp.iloc[res[i]]["assigned_skin_type"])


Produs 1: ELEVATE Advanced Retinol Serum
Produs  2 Superkind Fragrance-Free Bio-Barrier Serum Tip piele dry
Produs  3 Eye Doctor Moisture Care For Skin Around Eyes Tip piele combination
Produs  4 Mineral Sun Defense Moisturizer Broad Spectrum SPF 30 Tip piele combination
Produs  5 GENIUS Ultimate Anti-Aging Vitamin C+ Serum Tip piele combination
Produs  6 Glow Peel Pads Tip piele combination
Produs  7 C5 Super Boost Vitamin C Moisturizer Tip piele combination
Produs  8 Mini Hydra Vizor Invisible Moisturizer Broad Spectrum SPF 30 Sunscreen with Niacinamide Tip piele dry
Produs  9 AA (Alguronic Acid) Barrier Serum Tip piele combination
Produs  10 Green Tea Hyaluronic Acid Hydrating Eye Cream Tip piele dry
Produs  11 Mini The Essence  Skincare Boosting Treatment Tip piele combination
Produs  12 Liquid Sun Shield Daily Brightening Mineral Sunscreen SPF 50 Tip piele combination
Produs  13 Lotus Anti-Aging Night Moisturizer Tip piele combination
Produs  14 Dramatically Different Moisturizing

In [28]:
print(df_product["secondary_category"].value_counts())

secondary_category
Moisturizers       444
Treatments         378
Cleansers          273
Eye Care           143
Masks              106
Sunscreen           81
Self Tanners        40
Wellness            31
High Tech Tools     10
Name: count, dtype: int64


In [32]:
pd.set_option('display.max_colwidth', None)
print(df_product.head(10)["highlights"])

89                       ['Vegan', 'Good for: Loss of firmness', 'Collagen', 'Hypoallergenic', 'Without Parabens', 'Best for Dry, Combo, Normal Skin']
90                       ['Vegan', 'Good for: Loss of firmness', 'Collagen', 'Hypoallergenic', 'Without Parabens', 'Best for Dry, Combo, Normal Skin']
93                                                             ['Vegan', 'Hypoallergenic', 'UV Protection', 'SPF', 'Best for Dry, Combo, Normal Skin']
94                                           ['Vegan', 'Collagen', 'Hypoallergenic', 'Good for: Loss of firmness', 'Best for Dry, Combo, Normal Skin']
100                                          ['Vegan', 'Collagen', 'Hypoallergenic', 'Good for: Loss of firmness', 'Best for Dry, Combo, Normal Skin']
104       ['Liquid Formula', 'Best for Oily, Combo, Normal Skin', 'Good for: Dullness/Uneven Texture', 'Good for: Dryness', 'Niacinamide', 'Oil Free']
105    ['Salicylic Acid', 'Collagen', 'Best for Oily, Combo, Normal Skin', 'Good for: Dullness

In [33]:
print(df_product.head(10)["ingredients"])

89                                                                                                                    ['Collagen (Vegan)*, Water (Aqua, Eau), Ethylhexyl Palmitate, Oryza Sativa (Rice) Bran Extract, Caprylic/Capric Triglyceride, Glycerin, Cetearyl Methicone, Dimethicone, Cetearyl Alcohol, Pyrus Malus (Apple) Fruit Extract, Chlorella Protothecoides Oil, Polysorbate 60, Glyceryl Glucoside , Polyglyceryl-2 Stearate, Parachlorella Beijerinckii Exopolysaccharides, Collagen Amino Acids (Vegan)*, Ceramide NP, Silybum Marianum Fruit Extract, Helianthus Annuus (Sunflower) Extract, Rosmarinus Officinalis (Rosemary) Leaf Extract, Tocopherol, Carnosine, Sodium Stearoyl Glutamate, Caprylyl Glycol, Ammonium Acryloyldimethyltaurate/VP Copolymer, Ethylhexylglycerin, Hexylene Glycol, Citric Acid, Lecithin, Sodium Hydroxide, Sodium Phytate, t-Butyl Alcohol, Leuconostoc/Radish Root Ferment Filtrate, Phenoxyethanol, Triethyl Citrate, Vanillin, Amyl Salicylate, Benzyl Acetate, Cedrus Atlanti